In [ ]:
%matplotlib inline


NLP From Scratch: Generating Names with a Character-Level RNN
*************************************************************
**Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>`_

这是我们第二份"NLP From Scratch"的tutorials。
在`第一份tutorial </intermediate/char_rnn_classification_tutorial>`中我们使用了RNN来对名字归类到它们原来的语言中。这次我们会从语言中生成名字。

::

    > python sample.py Russian RUS
    Rovakov
    Uantov
    Shavakov

    > python sample.py German GER
    Gerren
    Ereng
    Rosher

    > python sample.py Spanish SPA
    Salla
    Parer
    Allan

    > python sample.py Chinese CHI
    Chan
    Hang
    Iun
我们会通过手写线性层手改进小的RNN。跟以往最大的不同是不在阅读名字的所有字母后预测分类，我们同时输入一个分类和输出一个字母。不断地预测字符来组织语言(这个可以同时通过词语或者其他更高的循序结构)，这个是常常被当作“语言模型”。

**推荐阅读:**
我们假定你有至少安装PyTorch，懂得Python，以及懂得张量:

-  https://pytorch.org/ 安装指南
-  :doc:`/beginner/deep_learning_60min_blitz` 通过这个来开始PyTorch
-  :doc:`/beginner/pytorch_with_examples` 通过这个获得一个广而深的概览
-  :doc:`/beginner/former_torchies_tutorial` 如果你是之前Lua Torch用户

这对同时认识RNNs以及它们怎么工作的非常有用:

-  `RNN不合理的有效性 <https://karpathy.github.io/2015/05/21/rnn-effectiveness/>`__
   这里有很多例子可以说明
-  `理解LSTM神经网络<https://colah.github.io/posts/2015-08-Understanding-LSTMs/>`__
   这是特别地跟LSTMs有关但是同时充满RNNs信息的

我同时建议之前的tutorial,:文档:`/intermediate/char_rnn_classification_tutorial`


Preparing the Data
==================

.. 注意::
   从这里下载数据
   `这里<https://download.pytorch.org/tutorial/data.zip>`_
   把它抽取到目前的目录下。

在这里可以看到最后的tutorial中关于这个程序的更多细节。总的来说，有很多plain tex文件``data/names/[Language].txt``每一行中一个名字。我们把行分割进数组，转换Unicode到ASCII,然后用字典结束``{language: [names ...]}``。


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

Creating the Network
====================

这个网络扩展了`RNN最后一份tutorial<#Creating-the-Network>`__
类别张量使用了一个额外的语句，这是跟其他语句一起链接起来的。这个类别张量是类似字母输入的独热向量。

我们会把下一个字母的概率输出翻译出来。当抽样时，最可能作为下一个输入的是上一个输出字母。

我们增加了第二个线性层``o2o``(在组合隐藏层以及输出时)来给它更多的能量。这里同时包括dropout层，可以从 `它输入的随机归零部分<https://arxiv.org/abs/1207.0580>`__ 看到，带有一个给定的概率(这里是0.1)以及通常给模糊输入来预防过拟合。这里我们在神经网络最后的部分加入噪声增加样本多样性来使用它。

.. figure:: https://i.imgur.com/jzVrf7f.png
   :alt:

In [ ]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

Training
=========
Preparing for Training
----------------------

首先，使用helper函数来得到随机对 (类别, 行):

In [ ]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

对于每一个timestep(那就是，对于每一个在训练单词中的字母)，网络的输入会``(category, current letter, hidden state)`` 以及输出会``(next letter, next hidden state)``。


因此对于每一个训练集合来说，我们会需要类别，一个字母输入的集合，一个字母输出的集合。

因为我们从现在每一步timestep的字母预测下一个字母，这个字母对是在line里面一组又一组连续的字母。比如说``"ABCD<EOS>"``我们会创建("A", "B"), ("B", "C"),("C", "D"), ("D", "EOS")。

.. 例子:: https://i.imgur.com/JH58tXY.png
   :alt:
类别张量是一个 `独热张量 <https://en.wikipedia.org/wiki/One-hot>`__ 大小是
``<1 x n_categories>``。当训练时我们需要在每一步timestep喂给它 - 这是一个设计的设备, 它可以包括一部分初始隐藏状态或者一些其他策略。


In [ ]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

为了训练的方便我们制造了``randomTrainingExample``函数来获得随机(category, line)对以及把它们变成所必须的(category, input, target)张量。

In [ ]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

Training the Network
--------------------

与分类相反的是，分类仅仅使用了最后的输出，我们在每一步都有预测，因此我们每一步都有损失。

自动梯度的魔法在于允许你去简单地把这每一步的损失加总以及最后使用call back。

In [ ]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

为了跟踪训练时间，我增加了``timeSince(timestamp)``函数，它会返回人类可读的字符:

In [ ]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

训练跟商业一样 - 呼叫一大串训练时间以及等待一些时间，每一次例子用``print_every``打印目前的时间以及损失，然后用``plot_every``在之后打印图``all_losses``中保持储存平均损失。

In [ ]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

Plotting the Losses
-------------------

从all \ _losses 展示的神经网络学习中打印历史损失：

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

Sampling the Network
====================

为了给我们的神经网络一个字母以及要求它预测下一个是什么，喂给它下一个字母以及重复直到EOS token。

-  为输入category创建张量，开始字母，以及空隐藏状态
-  从开始字母创建一个字符``output_name``
-  到最大的输出长度

   -  把现在得到字母喂到神经网络
   -  把下一个单词从最大的输出中拿出来，然后就是下一个隐藏状态
   -  如果单词就是EOS，在那里停止
   -  如果是一个普通的字母，增加``output_name``然后继续

-  回到最后一个名字

.. 注意::
   不是通过给予它一个初始字母，另一个策略是包括一个“初始字符” token在训练以及有神经网络选择它作为初始字母。

In [ ]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUS')

samples('German', 'GER')

samples('Spanish', 'SPA')

samples('Chinese', 'CHI')

Exercises
=========

-  尝试用不同的分类数据集  ->  行，比如说:

   -  人造的序列 -> 字符名字
   -  部分演讲 -> 词语
   -  国家 -> 城市

-  使用一个"句子开始" token 这样抽样可以从选择一个初始字母开始
-  通过得到一个更大或者更好形状的网络来得到更好的结果

   -  尝试nn.LSTM以及nn.GRU层
   -  组合多种RNNs作为更高水平的网络